In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import random
import os
from sklearn.preprocessing import LabelEncoder
# Used to scale the data features
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
# Used to calculate AUC
from sklearn.metrics import roc_auc_score
import time

In [32]:
# Create a list of headings for the data.

# Read in the data from the Stanford website.
loan = pd.read_csv('Loan_default.csv')
loan.columns

Index(['LoanID', 'Age', 'Income', 'LoanAmount', 'CreditScore',
       'MonthsEmployed', 'NumCreditLines', 'InterestRate', 'LoanTerm',
       'DTIRatio', 'Education', 'EmploymentType', 'MaritalStatus',
       'HasMortgage', 'HasDependents', 'LoanPurpose', 'HasCoSigner',
       'Default'],
      dtype='object')

In [33]:
loan_data = loan.drop(['LoanID','Education','EmploymentType', 'MaritalStatus',
       'HasMortgage', 'HasDependents', 'LoanPurpose', 'HasCoSigner'],axis=1)

In [34]:
# Create a separate dataframe containing the features (X) and response(Y).
X = loan_data.iloc[:,:-1]
  # drops the last column of the dataframe that contains the iris type (response).
Y = loan.iloc[:,-1]

In [35]:
# scale
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [41]:
n = X.shape[0]
k = X.shape[1]
b = 1
l = int(n//b) -1
p = 50
h = 4
o = 1
r = 0.0001
h_weight = np.zeros((p,k,h))
o_weight = np.zeros((p,h,o))
h_bias = np.zeros((p,1,h))
o_bias = np.zeros((p,1,o))
h_node = np.zeros((l,h))
o_node = np.zeros((l,o))

In [ ]:
start_time = time.time()
for m in range(0,p,1):
  if m == 1:
    h_weight_init = np.random.rand(k,h)
    o_weight_init = np.random.rand(h,o)
    h_bias_init = np.random.rand(1,h)
    o_bias_init = np.random.rand(1,o)
    h_weight[0,:,:] = h_weight_init
    o_weight[0,:,:] = o_weight_init
    h_bias[0,:,:] = h_bias_init
    o_bias[0,:,:] = o_bias_init
  else:
    h_weight[m,:,:] = h_weight[m-1,:,:]
    o_weight[m,:,:] = o_weight[m-1,:,:]
    h_bias[m,:,:] = h_bias[m-1,:,:]
    o_bias[m,:,:] = o_bias[m-1,:,:]
  for i in range(0,l,1):
    pos = i * b
    X_batch = X[pos:pos+b,:]
    Y_batch = Y[pos:pos+b]
    e = np.zeros((b,5))
    for j in range(0,h):
      h_node[i,j] = np.sum(np.dot(X_batch,h_weight[m,:,j]) + h_bias[m,:,j])
      h_node[i,j] = 1/(1+np.exp(-h_node[i,j]))
    o_node[i] = np.sum(np.dot(h_node[i,:],o_weight[m,:,:]) + o_bias[m,:,:])
    o_node[i] = 1/(1+np.exp(-o_node[i]))
    e[:,0] = o_node[i]
    e[:,1] = Y_batch
    if e[:,0] > 0.5:
      e[:,2] = 1
    else:
      e[:,2] = 0
    if e[:,2] == e[:,1]:
      e[:,4] = 1
    else:
      e[:,4] = 0
    e[:,3] = (e[:,1] - e[:,0])**2
    error = np.sum(e[:,3])
    if i < l - 1:
      delta_bp = o_node[i] * (1 - o_node[i]) * (Y_batch - o_node[i])
      delta_h = h_node[i] * (1 - h_node[i]) * np.dot(o_weight[m,:,:],delta_bp)
      o_weight[m,:,:] = o_weight[m,:,:] + r * delta_bp.ravel() * h_node[i].reshape(-1,1)
      o_bias[m,:,:] = o_bias[m,:,:] + r * delta_bp.ravel()
      h_weight[m,:,:] = h_weight[m,:,:] + (r * delta_h.reshape(-1,1) * X_batch).T
      h_bias[m,:,:] = h_bias[m,:,:] + r * delta_h.reshape(-1,1).T
    # evaluate final result accuracy
    e = np.zeros((n,5))
    h_node_all = np.zeros((n,h))
    o_node_all = np.zeros((n,o))
  for i in range(0,n,1):
    h_node_all[i,:] = np.sum(np.dot(X[i,:],h_weight[m,:,:]) + h_bias[m,:,:])
    h_node_all[i,:] = 1/(1+np.exp(-h_node_all[i,:]))
    o_node_all[i] = np.sum(np.dot(h_node_all[i,:],o_weight[m,:,:]) + o_bias[m,:,:])
    o_node_all[i] = 1/(1+np.exp(-o_node_all[i]))
    e[i,0] = o_node_all[i]
    e[i,1] = Y[i]
    if e[i,0] > 0.5:
      e[i,2] = 1
    else:
      e[i,2] = 0
    if e[i,2] == e[i,1]:
      e[i,4] = 1
    else:
      e[i,4] = 0
    e[i,3] = (e[i,1] - e[i,0])**2
  error_all = np.sum(e[:,3])
  accuracy = np.sum(e[:,4])/n
  print("Epoch:",m)
  print("Accuracy:", accuracy)
  print("Total Error:", error)
print(int(time.time() - start_time)//60)